# Gravitational wave detection



In [1]:
import numpy as np # Import the NumPy library for numerical operations
from pathlib import Path # Import the Path class from the pathlib module to handle file system paths
from scipy.stats import norm # Import the norm class from scipy.stats for statistical functions

# Poner comentarios en cada linea de que significa cada cosa
# En alguna parte se asegura que los datos esten balanceados.
# Tarea: Generar datos con imbalance (75 señal, 25 ruido) (25-75) (90-10) (10-90)

# Define the main function `make_gravitational_waves`
def make_gravitational_waves(
    path_to_data: Path, # Path to the directory containing the data
    n_signals: int = 30, # Number of signals to generate
    downsample_factor: int = 2, # Factor by which to downsample the signals
    Rcoef: float = 0.5, # Signal-to-noise ratio (SNR) coefficient
    perc_signal: float = 0.5, # Data partitioning factor
        ):
    def padrand(V, n, kr):
        cut = np.random.randint(n) # Generate a random integer to determine the split point for padding
        rand1 = np.random.randn(cut) # Create random noise for the first part of the padding
        rand2 = np.random.randn(n - cut) # Create random noise for the second part of the padding
        
        # Concatenate the first padding, the input signal `V`, and the second padding
        # Scale the padding by the factor `kr`
        out = np.concatenate((rand1 * kr, V, rand2 * kr))
        return out

    Npad = 50  # number of padding points on either side of the vector
    gw = np.load("../data/gravitational_wave_signals.npy") # Load data
    Norig = len(gw["data"][0]) # Get the original number of data points in each signal
    Ndat = len(gw["signal_present"]) # Get the total number of signals in the dataset todo es uno
    N = int(Norig / downsample_factor) # Calculate the number of data points after downsampling

    # Initialize lists to store noise coefficients and SNR coefficients
    ncoeff = []
    Rcoeflist = []

    # Loop through the number of signals to generate noise coefficients and SNR coefficients
    for j in range(n_signals):
        # Calculate the noise coefficient based on the R coefficient
        ncoeff.append(10 ** (-19) * (1 / Rcoef))
        # Append the corresponding R coefficient to the list
        Rcoeflist.append(Rcoef)

    # Initialize variables
    noisy_signals = []
    gw_signals = []
    k = 0
    labels = np.zeros(n_signals)

    # Loop through the number of signals to generate noisy signals and labels
    for j in range(n_signals):
        # Select a signal from the dataset and downsample it
        signal = gw["data"][j % Ndat][range(0, Norig, downsample_factor)]
        
        # Randomly decide if the signal is present (1) or absent (0)
        crit_val = norm.ppf(perc_signal) # Critical value for the normal distributio
        sigp = int((np.random.randn() < crit_val))
        
        # Generate random noise scaled by the noise coefficient
        noise = ncoeff[j] * np.random.randn(N)
        
        # Assign the label based on whether the signal is present
        labels[j] = sigp
        
        # If the signal is present, add it to the noise and pad the result
        if sigp == 1:
            rawsig = padrand(signal + noise, Npad, ncoeff[j])
            
            # Ensure at least one signal is present in the dataset
            if k == 0:
                k = 1
        else:
            # If the signal is absent, pad only the noise
            rawsig = padrand(noise, Npad, ncoeff[j])
        
        # Append the padded noisy signal to the list
        noisy_signals.append(rawsig.copy())
        
        # Append the original signal to the list
        gw_signals.append(signal)
    
    # Return the generated noisy signals, original signals, and labels
    return noisy_signals, gw_signals, labels

In [2]:
# Dataset 50-50
# generate and get data
R = 0.50 # Maximum signal-to-noise ratio (SNR) coefficient
n_signals = 100 # Number of signals to generate
DATA = Path(".") # Path to the directory containing the data
dataPartition = 0.5 # Data partitioning factor
noisy_signals_50_50, gw_signals_50_50, labels_50_50 = make_gravitational_waves(
    path_to_data=DATA, n_signals=n_signals, Rcoef=R, perc_signal=dataPartition
)
noisy_signals_50_50 = np.array(noisy_signals_50_50) # Convert the list of noisy signals to a NumPy array
gw_signals_50_50 = np.array(gw_signals_50_50) # Convert the list of original signals to a NumPy array
labels_50_50 = np.array(labels_50_50) # Convert the labels list to a NumPy array

# Dataset 25-75
dataPartition = 0.25 # Data partitioning factor
noisy_signals_25_75, gw_signals_25_75, labels_25_75 = make_gravitational_waves(
    path_to_data=DATA, n_signals=n_signals, Rcoef=R, perc_signal=dataPartition
)
noisy_signals_25_75 = np.array(noisy_signals_25_75) # Convert the list of noisy signals to a NumPy array
gw_signals_25_75 = np.array(gw_signals_25_75) # Convert the list of original signals to a NumPy array
labels_25_75 = np.array(labels_25_75) # Convert the labels list to a NumPy array

# Dataset 75-25
dataPartition = 0.75 # Data partitioning factor
noisy_signals_75_25, gw_signals_75_25, labels_75_25 = make_gravitational_waves(
    path_to_data=DATA, n_signals=n_signals, Rcoef=R, perc_signal=dataPartition
)
noisy_signals_75_25 = np.array(noisy_signals_75_25) # Convert the list of noisy signals to a NumPy array
gw_signals_75_25 = np.array(gw_signals_75_25) # Convert the list of original signals to a NumPy array
labels_75_25 = np.array(labels_75_25) # Convert the labels list to a NumPy array

# Dataset 10-90
dataPartition = 0.10 # Data partitioning factor
noisy_signals_10_90, gw_signals_10_90, labels_10_90 = make_gravitational_waves(
    path_to_data=DATA, n_signals=n_signals, Rcoef=R, perc_signal=dataPartition
)
noisy_signals_10_90 = np.array(noisy_signals_10_90) # Convert the list of noisy signals to a NumPy array
gw_signals_10_90 = np.array(gw_signals_10_90) # Convert the list of original signals to a NumPy array
labels_10_90 = np.array(labels_10_90) # Convert the labels list to a NumPy array

# Dataset 90-10
dataPartition = 0.90 # Data partitioning factor
noisy_signals_90_10, gw_signals_90_10, labels_90_10 = make_gravitational_waves(
    path_to_data=DATA, n_signals=n_signals, Rcoef=R, perc_signal=dataPartition
)
noisy_signals_90_10 = np.array(noisy_signals_90_10) # Convert the list of noisy signals to a NumPy array
gw_signals_90_10 = np.array(gw_signals_90_10) # Convert the list of original signals to a NumPy array
labels_90_10 = np.array(labels_90_10) # Convert the labels list to a NumPy array

In [3]:
print(labels_50_50.sum()) # Print the sum of labels to check the number of signals present
print(labels_25_75.sum()) # Print the sum of labels to check the number of signals present
print(labels_75_25.sum()) # Print the sum of labels to check the number of signals present
print(labels_10_90.sum()) # Print the sum of labels to check the number of signals present
print(labels_90_10.sum()) # Print the sum of labels to check the number of signals present

56.0
31.0
75.0
6.0
89.0


In [4]:
len(noisy_signals_50_50[0])
time_delay = 2
dimension = 50
from gtda.time_series import TakensEmbedding
TE = TakensEmbedding(time_delay=time_delay, dimension=dimension)
data = TE.fit_transform(noisy_signals_50_50)

print(data.shape)
print(len(data))
print(noisy_signals_50_50.shape)
print(len(noisy_signals_50_50))
data[0]

(100, 8144, 50)
100
(100, 8242)
100


array([[-4.38484381e-20,  5.00892720e-19, -3.68882873e-20, ...,
        -7.10830301e-20, -4.51767246e-19, -1.00441488e-19],
       [-2.68558970e-19,  1.47136405e-19,  2.53630913e-19, ...,
         4.49784482e-20, -1.09226276e-19,  3.24602585e-20],
       [ 5.00892720e-19, -3.68882873e-20,  1.34254817e-19, ...,
        -4.51767246e-19, -1.00441488e-19, -7.18014334e-20],
       ...,
       [-1.47628102e-19, -9.70302848e-20,  3.34606202e-19, ...,
         5.66656927e-19, -1.88403063e-19,  7.90906152e-20],
       [-1.87465155e-19,  2.12781333e-19,  2.04979125e-19, ...,
        -1.27353305e-19,  1.89767329e-19, -4.66810392e-21],
       [-9.70302848e-20,  3.34606202e-19, -9.31628995e-20, ...,
        -1.88403063e-19,  7.90906152e-20,  2.33612470e-19]])

# Pipeline Creation

In [17]:
from gtda.time_series import SlidingWindow
from gtda.time_series import TakensEmbedding
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import Amplitude
from gtda.pipeline import make_pipeline
from sklearn import set_config
from sklearn.ensemble import RandomForestClassifier

time_delay = 30
dimension = 100
stride = 5

TE = TakensEmbedding(time_delay=time_delay, dimension=dimension, stride=stride)
VR = VietorisRipsPersistence() 
Ampl = Amplitude()
RFR = RandomForestClassifier(criterion="entropy", random_state=37)
pipe = make_pipeline(TE, VR, Ampl, RFR)
pipe

Pipeline(steps=[('takensembedding',
                 TakensEmbedding(dimension=100, stride=5, time_delay=30)),
                ('vietorisripspersistence', VietorisRipsPersistence()),
                ('amplitude', Amplitude()),
                ('randomforestclassifier',
                 RandomForestClassifier(criterion='entropy', random_state=37))])

In [18]:
pipe_50_50 = make_pipeline(TE, VR, Ampl, RFR)
pipe_50_50.fit(noisy_signals_50_50, labels_50_50)
pipe_50_50.score(noisy_signals_50_50, labels_50_50)

0.56

In [19]:
# pipe_50_50 metrics
from sklearn.metrics import classification_report, confusion_matrix

y_pred_50_50 = pipe_50_50.predict(noisy_signals_50_50)
print(classification_report(labels_50_50, y_pred_50_50))
print(confusion_matrix(labels_50_50, y_pred_50_50))


              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        44
         1.0       0.56      1.00      0.72        56

    accuracy                           0.56       100
   macro avg       0.28      0.50      0.36       100
weighted avg       0.31      0.56      0.40       100

[[ 0 44]
 [ 0 56]]


/home/cristobal/anaconda3/envs/gravitational_wave_analysis/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/cristobal/anaconda3/envs/gravitational_wave_analysis/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/cristobal/anaconda3/envs/gravitational_wave_analysis/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter t

In [12]:
pipe_25_75 = make_pipeline(TE, VR, Ampl, RFR)
pipe_25_75.fit(noisy_signals_25_75, labels_25_75)
pipe_25_75.score(noisy_signals_25_75, labels_25_75)

0.69

In [13]:
pipe_75_25 = make_pipeline(TE, VR, Ampl, RFR)
pipe_75_25.fit(noisy_signals_75_25, labels_75_25)
pipe_75_25.score(noisy_signals_75_25, labels_75_25)

0.75

In [14]:
pipe_10_90 = make_pipeline(TE, VR, Ampl, RFR)
pipe_10_90.fit(noisy_signals_10_90, labels_10_90)
pipe_10_90.score(noisy_signals_10_90, labels_10_90)

0.94

In [15]:
pipe_90_10 = make_pipeline(TE, VR, Ampl, RFR)
pipe_90_10.fit(noisy_signals_90_10, labels_90_10)
pipe_90_10.score(noisy_signals_90_10, labels_90_10)

KeyboardInterrupt: 